In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/Real_Filter.zip" -d "/content"

Archive:  /content/Real_Filter.zip
  inflating: /content/Real_Filter.mp4  


In [ ]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom',path='/content/drive/MyDrive/Machine_Learning/YOLO/yolov5/Object_detection/Vi_nhua/Full/weights/best.pt', force_reload=True)
model_Droplet = torch.hub.load('ultralytics/yolov5', 'custom',path='/content/drive/MyDrive/Machine_Learning/YOLO/yolov5/Object_detection/Vi_nhua/Droplet_thuan/weights/best.pt', force_reload=True)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
requirements: Ultralytics requirements ['gitpython>=3.1.30', 'Pillow>=10.0.1'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.

In [ ]:
# class 0 is cell, 1 is droplet
import math
import matplotlib.pyplot as plt

def invest_pp(img,para):
  use = img #.astype(np.float32)
  use = cv2.cvtColor(use,cv2.COLOR_RGB2BGR)
  use = cv2.resize(use,para,interpolation=cv2.INTER_CUBIC)
  return use

def predict(img,model,threshold = 0.5,param=(0,0)):
  detections = model(img)
  check_img = img.copy()
  count = 0
  results = detections.pandas().xyxy[0].to_dict(orient="records")
  #filter
  for result in results:
    conf = result['confidence']
    name = result['name']
    class_ = result['class']
    if name =='Plastic microbeads' and conf > 0.5:
      count+=1
      cv2.rectangle(check_img,(int(result['xmin']),int(result['ymin'])),(int(result['xmax']),int(result['ymax'])),(12,158,200),2)
  check_img = invest_pp(check_img,param)
  return count,check_img
def find_cnts(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(11,11),0)
    canny_cv = cv2.Canny(gray,650,1150,apertureSize=5,L2gradient= True)
    canny_cv = cv2.dilate(canny_cv, None, iterations=4)
    canny_cv = cv2.erode(canny_cv, None, iterations=3)
    cnts, hierarchy = cv2.findContours(canny_cv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    return cnts
    # return gray,canny_cv,cnts
def pp(img,sizee = (640,640)):
  use = img #.astype(np.float32)
  use = cv2.cvtColor(use,cv2.COLOR_BGR2RGB)
  use = cv2.resize(use,sizee,interpolation=cv2.INTER_CUBIC)
  # use = cv2.resize(use,sizee)
  return use
def vote(arr):
  arr = np.array(arr)
  counts = np.bincount(arr)
  # print(counts)
  # print(np.argmax(counts))
  return np.argmax(counts)

def get_info_Droplets(img,threshold = 0.5):
  use_img = pp(img)
  detections = model_Droplet(use_img)
  results = detections.pandas().xyxy[0].to_dict(orient="records")
  Arr_tensor = []
  for result in results:
    conf = result['confidence']
    name = result['name']
    class_ = result['class']
    if name =='ADroplet' and conf > 0.5:
      tensor = (int(result['xmin']),int(result['ymin']),int(result['xmax']),int(result['ymax']))
      cv2.rectangle(use_img,(int(result['xmin']),int(result['ymin'])),(int(result['xmax']),int(result['ymax'])),(12,158,200),2)
      Arr_tensor.append(tensor)
    img_new = invest_pp(use_img,(650,740))
  return Arr_tensor, len(Arr_tensor)


# **Dùng Xử lý ảnh Detect Droplets**

In [ ]:
import cv2
import numpy as np
from google.colab import drive
from sklearn import metrics
import pandas as pd
import os
from google.colab.patches import cv2_imshow
top_left_box = [50,200]#200
bottom_right_box = [138,500]#400
id = 0
video_path = '/content/1.mp4'
cap = cv2.VideoCapture(video_path)

cl = [(0,0,0),(0,255,0),(0,0,255)] #black, green, red
color_ = cl[2]
Arr_count = []
Sign_frameOrigin = True

frame_width = 180
frame_height = 700
frame_rate = 30


count_frame = 0


# Tạo đối tượng VideoWriter để lưu video
video_output_filename = '3_frame.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_output_filename, fourcc, frame_rate, (frame_width, frame_height))

if (cap.isOpened()== False):
    print("Error opening video file")
while cap.isOpened():
    ret, img = cap.read()
    if ret: # có ảnh là show
      frame_sub = img[200:900,800:980]#N_them
      cnts = find_cnts(frame_sub)
      # cv2.drawContours(frame_sub, cnts, -1, (0, 255, 0), 2)
      cv2.rectangle(frame_sub, (top_left_box[0],top_left_box[1]),(bottom_right_box[0],bottom_right_box[1]), color_, 2)
      if len(cnts) > 0:
        check = 0
        for i in range (0,len(cnts)-1):
          x_, y_, w_, h_ = cv2.boundingRect(cnts[i])
          if (y_ > top_left_box[1] and y_+h_ < bottom_right_box[1] and w_ > 10 and h_ > 10):
            cv2.rectangle(frame_sub, (x_,y_),(x_+w_,y_+h_), cl[1], 2)
            c_x = x_  + w_/2
            c_y = y_ + h_/2
            if (x_ > y_):
              x_new_1 = int(c_x - w_/2)
              y_new_1 = int(c_y - w_/2)
              x_new_2 = int(c_x + w_/2)
              y_new_2 = int(c_y + w_/2)
            else:
              x_new_1 = int(c_x - h_/2)
              y_new_1 = int(c_y - h_/2)
              x_new_2 = int(c_x + h_/2)
              y_new_2 = int(c_y + h_/2)

            # sub_img = frame_sub[y_:y_+h_,x_:x_+w_]
            sub_img = frame_sub[y_new_1:y_new_2,x_new_1:x_new_2]
            use_predImg = pp(sub_img)
            count,check_img = predict(use_predImg,model,param=(x_new_2-x_new_1,y_new_2-y_new_1))
            # print(count)
            # cv2_imshow(frame_sub)
            Arr_count.append(count)
            # cv2_imshow(frame_sub)
            # frame_sub[y_new_1:y_new_2,x_new_1:x_new_2] = check_img
            check = 1

        # print(check)
        if(check == 0 and Arr_count != []):
          Arr_count_8 = Arr_count[0:8]
          numberCells = vote(Arr_count_8)
          if len(Arr_count_8)>=8:
            cv2.putText(frame_sub, f'{numberCells}',(50,180),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),2, cv2.LINE_AA)
            print(Arr_count_8)
          Arr_count = []
          Sign_frameOrigin = True
      out.write(frame_sub)
      if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    else:
        break
cap.release()
out.release()

# **Dùng Model detect droplets**

In [ ]:
import cv2
import numpy as np
from google.colab import drive
from sklearn import metrics
import pandas as pd
import os
from google.colab.patches import cv2_imshow

top_left_box = [248,150]#200
bottom_right_box = [385,400]#400

id = 0
video_path = '/content/Real_Filter.mp4'
cap = cv2.VideoCapture(video_path)

cl = [(0,0,0),(0,255,0),(0,0,255)] #black, green, red
color_ = cl[2]
Arr_count = []
Sign_frameOrigin = True

frame_width = 640
frame_height = 640
frame_rate = 30

numberCells = 0

count_frame = 0

# Tạo đối tượng VideoWriter để lưu video
video_output_filename = '8_frame.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_output_filename, fourcc, frame_rate, (frame_width, frame_height))

if (cap.isOpened()== False):
    print("Error opening video file")
while cap.isOpened():
    ret, img = cap.read()
    if ret: # có ảnh là show
      frame_sub = img[250:900,525:1265]
      frame_sub = cv2.resize(frame_sub, (640, 640))
      img_sub = frame_sub.copy()#don't draw in here

      cv2.putText(frame_sub, f'{numberCells}',(260,180),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),2, cv2.LINE_AA)
      cv2.putText(frame_sub, f'{id}',(200,180),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),2, cv2.LINE_AA)

      cv2.rectangle(frame_sub, (top_left_box[0],top_left_box[1]),(bottom_right_box[0],bottom_right_box[1]), color_, 2)
      Arr_in4,len_Arrin4 = get_info_Droplets(img_sub)

      check = 0

      for i in range(0,len_Arrin4):
        x1,y1,x2,y2 = Arr_in4[i][0],Arr_in4[i][1],Arr_in4[i][2],Arr_in4[i][3]
        # print(Arr_in4[i])
        # print("And")
        if (y1 > top_left_box[1] and y2 < bottom_right_box[1]):
          cv2.rectangle(frame_sub, (x1,y1),(x2,y2), cl[1], 2)
          c_x = (x1+x2)/2
          c_y = (y1+y2)/2
          h_ = y2-y1
          x1_new = int(c_x - h_/2)
          x2_new = int(c_x + h_/2)
          y1_new = int(c_y - h_/2)
          y2_new = int(c_y + h_/2)
          use_predImg = img_sub[y1_new:y2_new,x1_new:x2_new]
          used_pred = pp(use_predImg)
          count,check_img = predict(used_pred,model,param=(x2_new-x1_new,y2_new-y1_new))
          # print(count)
          Arr_count.append(count)
          # cv2_imshow(use_predImg)
          check = 1

      # cv2_imshow(img)
      # print(len_Arrin4)

      if(check == 0 and Arr_count != []): # trong 1 frame khong co giot nao trong vung lam viec
        Arr_count = Arr_count[0:8]
        numberCells = vote(Arr_count)
        print(Arr_count)
        Arr_count = []
        id+=1
        print(id)
      out.write(frame_sub)
      if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    else:
        break
cap.release()
out.release()

[5, 5, 5, 5, 5, 5, 6, 6]
1
[3, 2, 2, 2, 2, 2, 2, 4]
2
[7, 7, 4, 8, 8, 8, 8, 7]
3
[0, 0, 0, 2, 2, 0, 0, 0]
4
[3, 3, 3, 4, 4, 4, 3, 3]
5
[1, 1, 1, 2, 2, 2, 1, 1]
6
[4, 4, 4, 5, 5, 4, 3, 3]
7
[0, 0, 0, 1, 0, 0, 0, 1]
8
[4, 4, 4, 3, 3, 4, 4, 4]
9
[1, 2, 2, 1, 1, 2, 2, 2]
10
[1, 1, 1, 1, 2, 1, 1, 2]
11
[2, 1, 1, 2, 2, 2, 3, 2]
12
[2, 2, 3, 2, 3, 3, 2, 4]
13
[2, 1, 1, 1, 1, 1, 2, 1]
14
[0, 1, 1, 0, 2, 1, 1, 0]
15
[1, 0, 0, 0, 1, 0, 0, 1]
16
[1, 1, 1, 2, 2, 2, 3, 2]
17
[0, 1, 0, 0, 1, 2, 1, 0]
18
[2, 2, 2, 2, 1, 2, 2, 1]
19
[2, 2, 2, 1, 2, 2, 2, 2]
20
[1, 1, 2, 1, 1, 0, 0, 1]
21
[2, 0, 0, 1, 1, 1, 0, 0]
22
[1, 1, 1, 1, 1, 0, 0, 0]
23
[2, 2, 2, 2, 1, 1, 1, 1]
24
[2, 2, 2, 2, 2, 2, 2, 2]
25
[2, 2, 1, 1, 1, 2, 1, 1]
26
[2, 2, 1, 1, 1, 1, 2, 2]
27
[0, 0, 0, 0, 0, 0, 1, 1]
28
[1, 1, 1, 1, 1, 1]
29
[0, 0, 0, 0, 1]
30
[0, 0, 0, 0, 0, 0, 0]
31
[0, 0, 0, 1, 0, 0]
32
[1, 1, 1, 1, 1, 1]
33
[1, 2, 1, 3, 3, 1]
34
[2, 1, 1, 1, 1, 1]
35
[0, 0, 0, 0, 0, 0]
36
[0, 0, 0, 0, 0, 0, 0]
37
[0, 0, 0, 0, 0, 0]
38
[0